In [ ]:
import mediapipe as mp
import cv2
import pandas as pd
import os

import math

def calculate_angle(point1, point2, point3):
    vector1 = (point1[0] - point2[0], point1[1] - point2[1])
    vector2 = (point3[0] - point2[0], point3[1] - point2[1])


    dot_product = vector1[0] * vector2[0] + vector1[1] * vector2[1]
    magnitude1 = math.sqrt(vector1[0] ** 2 + vector1[1] ** 2)
    magnitude2 = math.sqrt(vector2[0] ** 2 + vector2[1] ** 2)

    cosine_theta = dot_product / (magnitude1 * magnitude2)
    theta = math.acos(cosine_theta)

    angle_degrees = math.degrees(theta)

    return angle_degrees



mp_pose = mp.solutions.pose
qiyam_data = []
hip_threshold = 160
knee_threshold = 160

video_directory = 'C:/Users/syedn/OneDrive/Desktop/Islah.AI-FinalYearProject-ComputerVision/Qiyam_Videos'

with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    for video_file in os.listdir(video_directory):
        if video_file.endswith(".mp4"):
            video_path = os.path.join(video_directory, video_file)
            cap = cv2.VideoCapture(video_path)

            while cap.isOpened():
                ret, frame = cap.read()
                if not ret:
                    break

                frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                results = pose.process(frame_rgb)

                if results.pose_landmarks:
                
                    shoulder = results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER]
                    hip = results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_HIP]
                    knee = results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_KNEE]
                    ankle = results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_ANKLE]

                    shoulder_hip_angle = calculate_angle((shoulder.x, shoulder.y), (hip.x, hip.y), (ankle.x, ankle.y))
                    hip_knee_angle = calculate_angle((hip.x, hip.y), (knee.x, knee.y), (ankle.x, ankle.y))

                    
                    shoulder_hip_label = 'correct' if shoulder_hip_angle > hip_threshold else 'incorrect'
                    hip_knee_label = 'correct' if hip_knee_angle > knee_threshold else 'incorrect'

                    qiyam_data.append([shoulder_hip_angle, hip_knee_angle, shoulder_hip_label, hip_knee_label])

                if cv2.waitKey(1) & 0xFF == 27: 
                    break

    
            cap.release()


qiyam_df = pd.DataFrame(qiyam_data, columns=['Shoulder_Hip_Angle', 'Hip_Knee_Angle', 'Shoulder_Hip_Label', 'Hip_Knee_Label'])
qiyam_df.to_csv('qiyam_dataset.csv', index=False)
cv2.destroyAllWindows()


